### Program written by Pablo Sánchez-Palencia, 2022
Scope: To initialize the source .pkl file which will be used to save the predictions of the machine learning models for the targeted properties

In [1]:
### USER INPUT REQUIRED ###

# Windows path
repodir = r'C:\Users\pablo\OneDrive\Documentos\GitHub\GeSn2N4-ML'

In [2]:
# Import modules.
import pandas as pd
import os
import numpy as np
import time

In [3]:
#Number of structures and organization within the different composition folders
n_str=[1,2,31,186,762,1337,1291,515,97]
inv=[0,0.125,0.25,0.375,0.5,0.625,0.75,0.875,1]

In [4]:
#Load of all the necessary inputs

#CCF 
corr_mat=[]
with open(repodir + '\\repository_data\\correlation_matrix.txt', "r") as obj_file: 
    for line in obj_file: 
        corr_mat.append([int(item) for item in line.split()[1:]])  #We get rid of 1st element, the void element, no info in it.
tuples=list(zip(list(range(len(corr_mat))),corr_mat))
allstr=pd.DataFrame(tuples,columns=['id','CorrM'])
    
#Inversion degree    
y_inv=np.ones(len(allstr))
ind=0
for nconf,value in zip(n_str,inv):
    y_inv[ind:ind+nconf]*=value
    ind+=nconf
allstr['y']=y_inv

#Degeneracies of the structure
deg=[]
for file in os.listdir(repodir +'\\repository_data\\structure_files'):
    deg.append(file[file.index('w')+1:file.index('-')])
allstr['g']=deg
    
#Mixing Energy and Bandgaps if calculated   
gga_rep = pd.read_pickle(repodir + '\\descriptors\\input_data_cm.pkl')
gga_data=[]
data_mixe=[]
data_bge=[]
for compound in range(len(corr_mat)):
    if str(compound) in gga_rep['tag'].to_numpy():
        gga_data.append(True)
        data_mixe.append(gga_rep[gga_rep['tag']==str(compound)]['MixE'].item())
        data_bge.append(gga_rep[gga_rep['tag']==str(compound)]['HSE-corr'].item())
    else:
        gga_data.append(False)
        data_mixe.append(False)
        data_bge.append(False)
allstr['gga_data']=gga_data
allstr['MixE']=data_mixe
allstr['HSE-corr']=data_bge

In [5]:
# Save data frame to .pkl file.
allstr.to_pickle('str_corrm.pkl')